In [2]:
import duckdb
import random
import pandas as pd

In [3]:
PATH = '../data/fulldata_07_24_04_P_일반음식점.csv'
df = pd.read_csv(PATH, encoding='cp949')
df = df.copy()

C:\Users\PCN\AppData\Local\Temp\ipykernel_21908\3905001769.py:2: DtypeWarning: Columns (11,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, encoding='cp949')


In [4]:
def split_address(address):
    parts = address.split()
    
    if len(parts) >= 4:
        city_name = parts[0]  # 예: 전라북도
        district = parts[1]  # 예: 남원시
        town = parts[2]  # 예: 대강면
        road_name = ' '.join(parts[3:-1])  # 예: 대강월산길 (주소의 마지막 요소 전까지 모두 도로명으로 처리)
        bungee = parts[-1]  # 예: 37-16 (주소의 마지막 요소는 항상 번지로 처리)
        
        return city_name, district, town, road_name, bungee
    else:
        return None, None, None, None, None

In [5]:
def prepro_df(df):
    random_value = random.randint(10120, 10200)
    df = df[:random_value].loc[:, ['개방서비스명', '인허가일자', '폐업일자','영업상태명', '도로명전체주소', '사업장명', '업태구분명', '남성종사자수', '여성종사자수']]
    df['도로명전체주소'] = df['도로명전체주소'].fillna('')
    df['폐업일자'] = df['폐업일자'].fillna('운영중')
    df.loc[:, '도로명전체주소'] = [address.split(',')[0] for address in df['도로명전체주소']]

    df[['시도', '시군구', '읍면동', '도로명', '번지']] = df['도로명전체주소'].apply(lambda x: pd.Series(split_address(x)))
    df.loc[:, 'num'] = 1
    df = df.rename(columns={'도로명전체주소': '소재지', '사업장명': '시설명', '업태구분명': '구분'})
    df = df.reset_index().drop(columns='index')
    return df
    # df.to_csv(path, index=False)

In [6]:
sorted_day_df = df.sort_values(by='인허가일자',ascending=False)
sorted_day_df['인허가일자'] = pd.to_datetime(sorted_day_df['인허가일자'], format='%Y-%m-%d', errors='coerce')
sorted_day_df['인허가일자'].fillna(method='ffill', inplace=True)

df_2024 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2024].reset_index().drop('index', axis=1)
df_2023 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2023].reset_index().drop('index', axis=1)
df_2022 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2022].reset_index().drop('index', axis=1)
df_2021 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2021].reset_index().drop('index', axis=1)
df_2020 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2020].reset_index().drop('index', axis=1)

C:\Users\PCN\AppData\Local\Temp\ipykernel_21908\4174379415.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sorted_day_df['인허가일자'].fillna(method='ffill', inplace=True)
C:\Users\PCN\AppData\Local\Temp\ipykernel_21908\4174379415.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sorted_day_df['인허가일자'].fillna(method='ffill', inplace=True)


In [7]:
sorted_day_df

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장생산직직원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지,Unnamed: 47
1901785,1901786,일반음식점,07_24_04_P,5710000,5735000-101-2024-00154,2024-07-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,Y,198.28,NaN,NaN,NaN,NaN
1901639,1901640,일반음식점,07_24_04_P,3470000,3470000-101-2024-00304,2024-07-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,0.00,NaN,NaN,NaN,NaN
1901569,1901570,일반음식점,07_24_04_P,3230000,3230000-101-2024-00596,2024-07-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,34.00,NaN,NaN,NaN,NaN
1901570,1901571,일반음식점,07_24_04_P,3830000,3840000-101-2024-00130,2024-07-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,0.00,NaN,NaN,NaN,NaN
1901571,1901572,일반음식점,07_24_04_P,3830000,3840000-101-2024-00129,2024-07-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,0.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080180,2080181,일반음식점,07_24_04_P,3830000,3840000-101-1994-00153,1900-05-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,28.56,NaN,NaN,NaN,NaN
2080587,2080588,일반음식점,07_24_04_P,3830000,3850000-101-1993-00121,1900-05-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,75.33,NaN,NaN,NaN,NaN
2080781,2080782,일반음식점,07_24_04_P,3830000,3850000-101-1994-00045,1900-05-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,71.94,NaN,NaN,NaN,NaN
2128025,2128026,일반음식점,07_24_04_P,3830000,3840000-101-1997-00001,1900-05-31,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,Y,475.20,NaN,NaN,NaN,NaN


In [6]:
df_2024 = prepro_df(df_2024)
df_2023 = prepro_df(df_2023)
df_2022 = prepro_df(df_2022)
df_2021 = prepro_df(df_2021)
df_2020 = prepro_df(df_2020)

In [7]:
print(len(df_2020))
print(len(df_2021))
print(len(df_2022))
print(len(df_2023))
print(len(df_2024))

10200
10198
10131
10169
10136


In [21]:
df_2024[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2024-07-31,운영중,영업/정상,충청북도 청주시 청원구 율량로 192,상만이네야장포차,한식,0.0,0.0,충청북도,청주시,청원구,율량로,192,1
1,일반음식점,2024-07-31,운영중,영업/정상,대구광역시 달서구 상화로29길 34,집밥전문 찌개좀 먹어본놈,한식,0.0,0.0,대구광역시,달서구,상화로29길,,34,1
2,일반음식점,2024-07-31,운영중,영업/정상,서울특별시 송파구 백제고분로40길 36,진성쭈꾸미,한식,0.0,0.0,서울특별시,송파구,백제고분로40길,,36,1


In [22]:
df_2023[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2023-12-29,운영중,영업/정상,서울특별시 서대문구 이화여대길 72-6,굿슈베,기타,0.0,0.0,서울특별시,서대문구,이화여대길,,72-6,1
1,일반음식점,2023-12-29,운영중,영업/정상,충청남도 아산시 번영로43번길 20,석홍란아산칼국수&다도횟집,한식,0.0,0.0,충청남도,아산시,번영로43번길,,20,1
2,일반음식점,2023-12-29,운영중,영업/정상,전라남도 곡성군 곡성읍 읍내20길 16-1,푸른낙타,기타,0.0,0.0,전라남도,곡성군,곡성읍,읍내20길,16-1,1


In [23]:
df_2022[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2022-12-30,2023-02-12,폐업,전북특별자치도 남원시 운봉읍 바래봉길 214,눈꽃분식,분식,0.0,0.0,전북특별자치도,남원시,운봉읍,바래봉길,214,1
1,일반음식점,2022-12-30,운영중,영업/정상,강원특별자치도 강릉시 중앙시장2길 10-3,부부보쌈,한식,0.0,0.0,강원특별자치도,강릉시,중앙시장2길,,10-3,1
2,일반음식점,2022-12-30,운영중,영업/정상,강원특별자치도 원주시 미래로 25,모이세,한식,0.0,0.0,강원특별자치도,원주시,미래로,,25,1


In [24]:
df_2021[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2021-12-31,2024-02-22,폐업,전라남도 순천시 도장안길 34,일등수산,한식,0.0,0.0,전라남도,순천시,도장안길,,34,1
1,일반음식점,2021-12-31,운영중,영업/정상,인천광역시 남동구 소래역로 12,이모네 양념집,횟집,0.0,0.0,인천광역시,남동구,소래역로,,12,1
2,일반음식점,2021-12-31,2023-08-07,폐업,광주광역시 동구 무등로307번길 14-1,온정,기타,0.0,0.0,광주광역시,동구,무등로307번길,,14-1,1


In [25]:
df_2020[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2020-12-31,운영중,영업/정상,경상남도 진주시 정촌면 삼일로 12,가마치통닭정촌점,호프/통닭,NaN,NaN,경상남도,진주시,정촌면,삼일로,12,1
1,일반음식점,2020-12-31,운영중,영업/정상,광주광역시 광산구 첨단내촌로70번길 8-12,오늘,한식,NaN,NaN,광주광역시,광산구,첨단내촌로70번길,,8-12,1
2,일반음식점,2020-12-31,운영중,영업/정상,광주광역시 광산구 사암로390번길 13-32,짱구네분식,한식,NaN,NaN,광주광역시,광산구,사암로390번길,,13-32,1


In [8]:
df_2020.to_csv('../data/restaurant_2020.csv', index=False)
df_2021.to_csv('../data/restaurant_2021.csv', index=False)
df_2022.to_csv('../data/restaurant_2022.csv', index=False)
df_2023.to_csv('../data/restaurant_2023.csv', index=False)
df_2024.to_csv('../data/restaurant_2024.csv', index=False)

In [9]:
conn = duckdb.connect('../data/database.db') # database 연결

In [17]:
# 데이터 삭제
conn.execute('DROP TABLE IF EXISTS restaurant_2024')
conn.execute('DROP TABLE IF EXISTS restaurant_2023')
conn.execute('DROP TABLE IF EXISTS restaurant_2022')
conn.execute('DROP TABLE IF EXISTS restaurant_2021')
conn.execute('DROP TABLE IF EXISTS restaurant_2020')

In [18]:
# 데이터 저장
conn.execute('CREATE TABLE restaurant_2024 AS SELECT * FROM df_2024')
conn.execute('CREATE TABLE restaurant_2023 AS SELECT * FROM df_2023')
conn.execute('CREATE TABLE restaurant_2022 AS SELECT * FROM df_2022')
conn.execute('CREATE TABLE restaurant_2021 AS SELECT * FROM df_2021')
conn.execute('CREATE TABLE restaurant_2020 AS SELECT * FROM df_2020')

In [19]:
# database close
conn.close()